<a href="https://colab.research.google.com/github/EmiliaFidler/Intro_to_Comp_Ling_WS24/blob/main/final_project_comp_ling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Final Project**
---


*VU Introduction to Computational Linguistics - WS24/25*

---
**Topic**: Toxicity Detection in Social Media  
Detecting toxic or offensive language in social media, e.g. YouTube comments in Polish and English    

**Datasets**: PolishCyberbullyingDataset or hatecheck-polish for the Polish dataset and civil_comments or toxicity_classification_jigsaw for the English dataset
model: XLM-RoBERTa Base   



**Set-Up:**
Installing librarires required for the task

In [1]:
!pip install transformers
!pip install datasets
!pip install torch
!pip install scikit-learn
!pip install matplotlib seaborn
!pip install optuna
!pip install tensorboard
!pip install pandas
!pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 19.9 MB/s eta 0:00:00

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
from datasets import load_dataset, DatasetDict

# English dataset (civil_comments)
dataset_en = load_dataset("civil_comments")

# Polish dataset (PolishCyberbullyingDataset)
dataset_pl = load_dataset("ptaszynski/PolishCyberbullyingDataset")

README.md:   0%|          | 0.00/7.73k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/194M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/187M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.8M [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

v2_training_anonymized.tsv:   0%|          | 0.00/801k [00:00<?, ?B/s]

v2_test_anonymized.tsv:   0%|          | 0.00/79.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10038 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [14]:
print(dataset_en)
print(dataset_pl)

DatasetDict({
    train: Dataset({
        features: ['text', 'toxicity', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack', 'sexual_explicit'],
        num_rows: 1804874
    })
    validation: Dataset({
        features: ['text', 'toxicity', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack', 'sexual_explicit'],
        num_rows: 97320
    })
    test: Dataset({
        features: ['text', 'toxicity', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack', 'sexual_explicit'],
        num_rows: 97320
    })
})
DatasetDict({
    train: Dataset({
        features: ['TEXT', 'GENERAL TAG', 'SPECIFIC TAG', 'CATEGORIES', 'ANY PROFANITIES?'],
        num_rows: 10038
    })
    test: Dataset({
        features: ['TEXT', 'GENERAL TAG', 'SPECIFIC TAG', 'CATEGORIES', 'ANY PROFANITIES?'],
        num_rows: 1000
    })
})


In [15]:
train_valid_split = dataset_pl['train'].train_test_split(test_size=0.1, seed=42)

In [16]:
train_en = dataset_en['train']
val_en = dataset_en['validation']
test_en = dataset_en['test']

train_pl = train_valid_split['train']
val_pl = train_valid_split['test']
test_pl = dataset_pl['test']

In [17]:
print("Train size:", len(train_pl))
print("Validation size:", len(val_pl))
print("Test size:", len(test_pl))

print("Train size:", len(train_en))
print("Validation size:", len(val_en))
print("Test size:", len(test_en))

Train size: 9034
Validation size: 1004
Test size: 1000
Train size: 1804874
Validation size: 97320
Test size: 97320


Preprocessing!

In [18]:
from transformers import AutoTokenizer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def tokenize_function(examples):
  input_column = 'text' if 'text' in examples else 'TEXT'
  return tokenizer(examples[input_column], truncation=True, padding="max_length", max_length=128, return_tensors="pt")

# tokenizing dataset
tokenized_train_en = train_en.map(tokenize_function, batched=True)
tokenized_val_en = val_en.map(tokenize_function, batched=True)
tokenized_test_en = test_en.map(tokenize_function, batched=True)

tokenized_train_pl = train_pl.map(tokenize_function, batched=True)
tokenized_val_pl = val_pl.map(tokenize_function, batched=True)
tokenized_test_pl = test_pl.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # for dynamic padding during training

Map:   0%|          | 0/1804874 [00:00<?, ? examples/s]

Map:   0%|          | 0/97320 [00:00<?, ? examples/s]

Map:   0%|          | 0/97320 [00:00<?, ? examples/s]

Map:   0%|          | 0/9034 [00:00<?, ? examples/s]

Map:   0%|          | 0/1004 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# Define save paths in your Drive
save_path = '/content/drive/MyDrive/tokenized_datasets/'

# Save English datasets
tokenized_train_en.save_to_disk(save_path + 'tokenized_train_en')
tokenized_val_en.save_to_disk(save_path + 'tokenized_val_en')
tokenized_test_en.save_to_disk(save_path + 'tokenized_test_en')

# Save Polish datasets
tokenized_train_pl.save_to_disk(save_path + 'tokenized_train_pl')
tokenized_val_pl.save_to_disk(save_path + 'tokenized_val_pl')
tokenized_test_pl.save_to_disk(save_path + 'tokenized_test_pl')

Saving the dataset (0/4 shards):   0%|          | 0/1804874 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/97320 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/97320 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/9034 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1004 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [19]:
# loading my datasets
from datasets import load_from_disk

# Load English datasets
tokenized_train_en = load_from_disk('/content/drive/MyDrive/tokenized_datasets/tokenized_train_en')
tokenized_val_en = load_from_disk('/content/drive/MyDrive/tokenized_datasets/tokenized_val_en')
tokenized_test_en = load_from_disk('/content/drive/MyDrive/tokenized_datasets/tokenized_test_en')

# Load Polish datasets
tokenized_train_pl = load_from_disk('/content/drive/MyDrive/tokenized_datasets/tokenized_train_pl')
tokenized_val_pl = load_from_disk('/content/drive/MyDrive/tokenized_datasets/tokenized_val_pl')
tokenized_test_pl = load_from_disk('/content/drive/MyDrive/tokenized_datasets/tokenized_test_pl')

finetuning XLM-RoBERT Base

In [20]:
pip install accelerate

In [21]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load XLM-RoBERTa tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
model = AutoModelForSequenceClassification.from_pretrained(
    "xlm-roberta-base",
    num_labels=2  # Binary classification
)

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TrainingArguments --> for better control of training behaviour

In [22]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",           # Directory to save model and logs
    evaluation_strategy="epoch",     # Evaluate at the end of each epoch
    save_strategy="epoch",           # Save model at each epoch
    learning_rate=2e-5,              # Recommended for large pre-trained models
    per_device_train_batch_size=16,  # Adjust based on GPU memory
    per_device_eval_batch_size=16,   # Same as train batch size
    num_train_epochs=5,              # Start with 5 epochs, tune as needed
    weight_decay=0.01,               # Regularization
    logging_dir="./logs",            # Directory for logs
    logging_steps=10,                # Log training progress every 10 steps
    load_best_model_at_end=True,     # Load the best model during evaluation
    metric_for_best_model="accuracy", # Metric to select the best model
    save_total_limit=2               # Keep only 2 checkpoints to save space
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [27]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.8 MB/s eta 0:00:00


In [28]:
import evaluate

# Load accuracy metric
accuracy = evaluate.load("accuracy")

# Define compute_metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

In [24]:
# for handling of variable-length sequences in batchesv
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [29]:
from transformers import Trainer

trainer = Trainer(
    model=model,                       # Pre-trained model
    args=training_args,                # Training arguments
    train_dataset=tokenized_train_en,  # Training data
    eval_dataset=tokenized_val_en,     # Validation data
    tokenizer=tokenizer,               # Tokenizer
    data_collator=data_collator,       # Data collator for padding
    compute_metrics=compute_metrics    # Evaluation metric
)

<ipython-input-29-28902efdd03c>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
results = trainer.evaluate(tokenized_test_en)
print("Test Results:", results)